In [1]:
print("hello_world")

hello_world


In [1]:
import requests
import re
import os
from urllib.parse import urlparse, parse_qs

def download_vsix(marketplace_url):
    try:
        # 1. URL에서 Publisher와 Extension Name 추출
        parsed_url = urlparse(marketplace_url)
        query_params = parse_qs(parsed_url.query)

        if 'itemName' not in query_params:
            print("❌ 올바른 Marketplace URL이 아닙니다. (itemName 파라미터가 없습니다)")
            return

        full_item_name = query_params['itemName'][0] # 예: GrapeCity.gc-excelviewer

        if '.' not in full_item_name:
             print("❌ 올바른 형식의 itemName이 아닙니다.")
             return

        publisher, extension_name = full_item_name.split('.', 1)

        print(f"🔍 분석 중...")
        print(f"   - 게시자(Publisher): {publisher}")
        print(f"   - 확장명(Extension): {extension_name}")

        # 2. 최신 버전 정보 가져오기 (HTML 파싱)
        # 페이지 내의 메타데이터에서 버전을 찾습니다.
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(marketplace_url, headers=headers)

        if response.status_code != 200:
            print("❌ 페이지에 접속할 수 없습니다.")
            return

        # 정규표현식으로 버전 찾기 (JSON 데이터 내의 Version 키 값 추출)
        # 패턴: "Version","value":"4.2.64" 형태를 찾음
        version_pattern = r'"key":"Version","value":"([\d\.]+)"'
        match = re.search(version_pattern, response.text)

        if match:
            version = match.group(1)
            print(f"   - 최신 버전(Version): {version} (자동 감지됨)")
        else:
            # 자동 감지 실패 시 수동 입력 요청
            print("⚠️ 버전을 자동으로 찾을 수 없습니다.")
            version = input("👉 다운로드할 버전을 직접 입력해주세요 (예: 1.0.0): ").strip()

        # 3. 다운로드 링크 생성 (사용자가 제공한 공식 적용)
        download_url = (
            f"https://{publisher}.gallery.vsassets.io/_apis/public/gallery/publisher/"
            f"{publisher}/extension/{extension_name}/{version}/assetbyname/Microsoft.VisualStudio.Services.VSIXPackage"
        )

        # 파일명 생성
        filename = f"{publisher}.{extension_name}-{version}.vsix"

        print(f"\n🚀 다운로드 시작: {filename}")
        print(f"   URL: {download_url}")

        # 4. 파일 다운로드
        file_response = requests.get(download_url, stream=True)

        if file_response.status_code == 200:
            with open(filename, 'wb') as f:
                for chunk in file_response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"\n✅ 다운로드 완료! 파일이 현재 폴더에 저장되었습니다.")
            print(f"📂 파일명: {filename}")
        else:
            print(f"\n❌ 다운로드 실패 (상태 코드: {file_response.status_code})")
            print("   존재하지 않는 버전이거나 링크 형식이 잘못되었을 수 있습니다.")

    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")

if __name__ == "__main__":
    print("--- VS Code Extension(.vsix) 다운로더 ---")
    url = input("🔗 Marketplace URL을 입력하세요: ").strip()
    download_vsix(url)

--- VS Code Extension(.vsix) 다운로더 ---
🔍 분석 중...
   - 게시자(Publisher): ms-vscode
   - 확장명(Extension): remote-server
⚠️ 버전을 자동으로 찾을 수 없습니다.

🚀 다운로드 시작: ms-vscode.remote-server-1.6.2026020909.vsix
   URL: https://ms-vscode.gallery.vsassets.io/_apis/public/gallery/publisher/ms-vscode/extension/remote-server/1.6.2026020909/assetbyname/Microsoft.VisualStudio.Services.VSIXPackage

✅ 다운로드 완료! 파일이 현재 폴더에 저장되었습니다.
📂 파일명: ms-vscode.remote-server-1.6.2026020909.vsix
